In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools
# from test_utils import *
import test_utils
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
Lx, Lt = 4, 8
# Lx, Lt = 2, 8
L = Lattice(Lx, Lt)

In [28]:
# test_file = '/Users/poare/Dropbox (MIT)/research/0nubb/tests/hdf5/free_field_qlua_clover.h5'
test_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/tests/hdf5/free_field_qlua_clover.h5'
f1 = h5py.File(test_file, 'r')
Sk1_clover = f1['prop_k1/q2200'][()]
Sk2_clover = f1['prop_k2/q2200'][()]
Sq_clover = np.einsum('ijab->aibj', f1['prop_q/q2200'][()])
Sq_clover_b = np.expand_dims(Sq_clover, axis = 0)
Sq_clover_inv = invert_props(Sq_clover_b, Nb = 1)

In [30]:
# chroma_file = '/Users/poare/Dropbox (MIT)/research/0nubb/tests/hdf5/cfgEXAMPLE.h5'
chroma_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/tests/hdf5/cfgEXAMPLE.h5'
f2 = h5py.File(chroma_file, 'r')
Sk1_chroma = f2['prop_k1/q2200'][()]
Sk2_chroma = f2['prop_k2/q2200'][()]
Sq_chroma = f2['prop_q/q2200'][()]

Sq_chroma = np.einsum('ijab->aibj', f2['prop_q/q2200'][()])
Sq_chroma_b = np.expand_dims(Sq_chroma, axis = 0)
Sq_chroma_inv = invert_props(Sq_chroma_b, Nb = 1)

In [47]:
# test Zq-- why is it so small for chroma output?
bvec = np.array([0.0, 0.0, 0.0, 0.5], dtype = np.float32)
k1 = np.array([-2, 0, 2, 0])
k2 = np.array([0, 2, 2, 0])
q = k2 - k1
q_lat = np.sin(L.to_linear_momentum(q + bvec))
print('clover')
Zq_clover = quark_renorm(Sq_clover_inv, q_lat, Nb = 1)

print('chroma dwf')
Zq_chroma = quark_renorm(Sq_chroma_inv, q_lat, Nb = 1)

clover
(1.1668295954251867e-17+1.538274367229677e-17j)
(-9.2741463525386e-19+4.263717749566398e-17j)
1.9200068408580393e-17j
(3.599284038483899e-17-4.592201113700867j)
chroma dwf
(-2.1676845257845203e-15+5.698504679299543e-16j)
(-1.6122263652382782e-15+3.598416717898146e-16j)
(-2.4689644148318934e-14-1.7053025658242404e-13j)
(5.900304756778496e-06+5.4513126190825975e-15j)


In [48]:
Zq_clover

array([0.99999994+7.8378175e-18j], dtype=complex64)

In [49]:
Zq_chroma

array([-1.1870804e-15+1.2848532e-06j], dtype=complex64)

## Play around with free field output, written explicitly

In [ ]:
# Load in the free field propagators
job_num = 30558
# job_num = 30562
# k1 = np.array([-1, 0, 1, 0])
# k2 = np.array([0, 1, 1, 0])
k1 = np.array([-2, 0, 2, 0])
k2 = np.array([0, 2, 2, 0])
q = k2 - k1
q_lat = np.sin(L.to_linear_momentum(q + bvec))
qlat_slash = slash(q_lat)
test_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/free_field_props_' + str(job_num) + '/free_field_props.h5'
# test_file = '/Users/poare/Dropbox (MIT)/research/0nubb/tests/hdf5/free_field_qlua_clover.h5'
prop_k1, prop_k2, prop_q = test_utils.read_propagators(test_file, L, q)

In [ ]:
# Fourier transform and momentum project
FTk1 = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTk2 = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTq = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTmq = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTm2q = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
for x, y, z in itertools.product(range(Lx), repeat = 3):
    for t in range(Lt):
        pos = np.array([x, y, z, t])
        FTk1[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, k1 + bvec))
        FTk2[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, k2 + bvec))
        FTq[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, q + bvec))
        FTmq[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, -q))
        FTm2q[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, -2 * q))

In [ ]:
Sk1 = np.einsum('xyzt,xyzt...->...', FTk1, prop_k1) / V
Sk2 = np.einsum('xyzt,xyzt...->...', FTk2, prop_k2) / V
Sq = np.einsum('xyzt,xyzt...->...', FTq, prop_q) / V
antiprop_k2 = np.einsum('ik,...akbl,lj->...aibj', gamma5, adjoint(prop_k2), gamma5)
Sk1_inv = np.linalg.tensorinv(Sk1)
Sk2_inv = np.linalg.tensorinv(Sk2)
Sq_inv = np.linalg.tensorinv(Sq)

## Verify that Zq, ZV, and ZA are all equal to 1 for the free field

In [ ]:
# Verify Zq is 1
Zq = (1j) * sum([q[mu] * np.einsum('ij,ajai', analysis.gamma[mu], Sq_inv) for mu in range(analysis.d)]) \
    / (12 * analysis.square(q))
print(Zq)    # for free field, Zq must be 1.

In [ ]:
# Get and amputate three point functions
GV = np.array([np.einsum('xyzt,xyztaick,kl,xyztclbj->aibj', FTmq, antiprop_k2, analysis.gamma[mu], prop_k1) for mu in range(4)]) / V
GA = np.array([np.einsum('xyzt,xyztaick,km,ml,xyztclbj->aibj', FTmq, antiprop_k2, analysis.gamma[mu], analysis.gamma5, prop_k1) for mu in range(4)]) / V
ΓV = np.array([np.einsum('aibj,bjck,ckdl->aidl', Sk2_inv, GV[mu], Sk1_inv) for mu in range(4)])
ΓA = np.array([np.einsum('aibj,bjck,ckdl->aidl', Sk2_inv, GA[mu], Sk1_inv) for mu in range(4)])
qΓV = np.einsum('m,maibj->aibj', q_lat, ΓV)
qΓA = np.einsum('m,maibj->aibj', q_lat, ΓA)

In [ ]:
# Verify axial and vector R.C.s are 1
ZV = 12 * Zq * analysis.square(q_lat) / np.einsum('aiaj,ji->', qΓV, qlat_slash)
ZA = 12 * Zq * analysis.square(q_lat) / np.einsum('aiaj,jk,ki->', qΓA, analysis.gamma5, qlat_slash)
print(ZV)
print(ZA)

## Play around with the propagators and find some way to get the amputated 4 point function equal to its tree level value.

In [ ]:
# Confirm that I'm getting the right combinations of gamma matrices
A = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
# An_contr = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
GO = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
ΓO = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
for n in range(16):
    print(n)
    A[n] = np.einsum('...aick,kl,...clbj->...aibj', antiprop_k2, analysis.gammaList[n], prop_k1)
    An_contr = np.einsum('...ckdl,...aibj->...aibjckdl', A[n], A[n]) - np.einsum('...ckbj,...aidl->...aibjckdl', A[n], A[n])
    GO[n] = 2 * np.einsum('xyzt,xyzt...->...', FTm2q, An_contr) / V    # TODO later try with no momentum
    ΓO[n] = np.einsum('aiem,gobj,ckfn,hpdl,emgofnhp->aibjckdl', Sk2_inv, Sk1_inv, Sk2_inv, Sk1_inv, GO[n])  #z is n

In [ ]:
# Use reverse gamma matrices that Phiala uses
AR = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
# An_contrR = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
GOR = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
ΓOR = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
for n in range(16):
    print(n)
    AR[n] = np.einsum('...aick,kl,...clbj->...aibj', antiprop_k2, analysis.gammaListRev[n], prop_k1)
    An_contrR = np.einsum('...ckdl,...aibj->...aibjckdl', AR[n], AR[n]) - np.einsum('...ckbj,...aidl->...aibjckdl', AR[n], AR[n])
    GOR[n] = 2 * np.einsum('xyzt,xyzt...->...', FTm2q, An_contrR) / V    # TODO later try with no momentum
    ΓOR[n] = np.einsum('aiem,gobj,ckfn,hpdl,emgofnhp->aibjckdl', Sk2_inv, Sk1_inv, Sk2_inv, Sk1_inv, GOR[n])  #z is n

In [ ]:
for m in range(16):
    if np.allclose(A[m], AR[m]):
        print('n = ' + str(m) + ', +')
    elif np.allclose(A[m], -AR[m]):
        print('n = ' + str(m) + ', -')

In [ ]:
for m in range(16):
    if np.allclose(GO[m], GOR[m]):
        print('n = ' + str(m) + ', +')
    elif np.allclose(GO[m], -GOR[m]):
        print('n = ' + str(m) + ', -')
    else:
        print('n = ' + str(m) + ', no match')

In [ ]:
SS = ΓO[0]
PP = ΓO[15]
VV = ΓO[1] + ΓO[2] + ΓO[4] + ΓO[8]
# AA = -ΓO[14] + ΓO[13] - ΓO[11] + ΓO[7]    # SOMETHING WRONG WITH AA
# AA = -(ΓO[14] + ΓO[13] + ΓO[11] + ΓO[7])
AA = ΓO[14] + ΓO[13] + ΓO[11] + ΓO[7]
TT = (ΓO[3] + ΓO[5] + ΓO[6] + ΓO[9] + ΓO[10] + ΓO[12])

# SS = ΓOR[0]
# PP = ΓOR[15]
# VV = ΓOR[1] + ΓOR[2] + ΓOR[4] + ΓOR[8]
# AA = ΓOR[14] + ΓOR[13] + ΓOR[11] + ΓOR[7]
# TT = (ΓOR[3] + ΓOR[5] + ΓOR[6] + ΓOR[9] + ΓOR[10] + ΓOR[12])

eps = 1e-5
SS[np.abs(SS) < eps] = 0
PP[np.abs(PP) < eps] = 0
VV[np.abs(VV) < eps] = 0
AA[np.abs(AA) < eps] = 0
TT[np.abs(TT) < eps] = 0
# Gamma_D = np.einsum('aiajckcl->ijkl', Gamma)

In [ ]:
# [a, i, b, j, c, k, d, l]
# SS[:, 0, :, 0, :, 1, :, 1]

In [ ]:
Gamma = [VV + AA, VV - AA, SS - PP, SS + PP, TT]
P = analysis.projectors()
Lambda = np.einsum('nbjaidlck,maibjckdl->mn', P, Gamma)
F = analysis.getF()

In [ ]:
print(F)

In [ ]:
print(Lambda.astype(int))

In [ ]:
for i in range(5):
    print(np.einsum('nbjaidlck,aibjckdl->n', P, Gamma[i]))

In [ ]:
print(Lambda)

In [ ]:
print(Lambda.astype(int))    # note this truncates so some may be 1 less than in F

In [ ]:
F

In [ ]:
# Check out what the VV and AA amputated 4-pt functions actually look like
VV_ijkl = VV[0, :, 0, :, 1, :, 1, :]#.astype(int)
AA_ijkl = AA[0, :, 0, :, 1, :, 1, :]#.astype(int)

In [ ]:
print(np.allclose(2 * np.einsum('mij,mkl->ijkl', analysis.gamma, analysis.gamma), VV_ijkl))

In [ ]:
gamGam5 = np.array([analysis.gamma[mu] @ analysis.gamma5 for mu in range(4)])
print(np.allclose(2 * np.einsum('mij,mkl->ijkl', gamGam5, gamGam5), AA_ijkl))
# 2 * np.einsum('mij,mkl->ijkl', gamGam5, gamGam5) == AA_ijkl.astype(int)

In [ ]:
2 * np.einsum('mij,mkl->ijkl', gamGam5, gamGam5)

In [ ]:
AA_ijkl

## Compare with QLUA output. There seems to be a bug somewhere, since the 3-point functions don't match

In [ ]:
# Saving mqFT2, A_Gamma, and the Green's function. Compare all of them to what this code outputs. 
# when everything here matches, I should be able to feed QLUA output into free_field.py and have it work
qlua_jobnum = 30661
qlua_out_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/'+'free_field_'+str(qlua_jobnum)+'/free_field.h5'
fq = h5py.File(qlua_out_file, 'r')
mqFT2 = fq['mqFT2'][()]
A_gamma = np.einsum('...ijab->...aibj', np.array([fq['Agamma' + str(n)][()] for n in range(16)]))
qlua_GO = np.einsum('nijklabcd->naibjckdl', np.array([fq['Gn' + str(n) + '/q1100'][()] for n in range(16)]))

In [ ]:
# Compare FT factors from QLUA with Python
# print(mqFT2)
# print(FTm2q)
print(np.allclose(mqFT2, FTm2q))

In [ ]:
# Compare A_gamma output from QLUA and Python for some n.
n = 3
print(A_gamma[n])
print(A[n])

In [ ]:
# Compare QLUA GO output with Python GO output
# print(qlua_GO[n])
# print(GO[n])    # python output for GO at gamma = 3
print(np.allclose(qlua_GO[n], GO[n]))
print(np.allclose(np.einsum('aibjckdl->dlckbjai', qlua_GO[n]), GO[n]))